In [1]:
import ast
import syft as sy
import re
import biwrap
from syft.core.node.common.action.function_or_constructor_action import (
    RunFunctionOrConstructorAction,
)
from syft.core.common.uid import UID
from syft.core.io.address import Address
from syft.core.node.common.action.run_class_method_action import RunClassMethodAction
from syft.core.pointer.pointer import Pointer

In [2]:
# use new syckle secure custom code delivery and remote execution

In [3]:
domain = sy.Domain(name="Bob")
root_client = domain.get_root_client()

In [4]:
import inspect, sys

def new_getfile(object, _old_getfile=inspect.getfile):
    if not inspect.isclass(object):
        return _old_getfile(object)
    
    # Lookup by parent module (as in current inspect)
    if hasattr(object, '__module__'):
        object_ = sys.modules.get(object.__module__)
        if hasattr(object_, '__file__'):
            return object_.__file__
    
    # If parent module is __main__, lookup by methods (NEW)
    for name, member in inspect.getmembers(object):
        if inspect.isfunction(member) and object.__qualname__ + '.' + member.__name__ == member.__qualname__:
            return inspect.getfile(member)
    else:
        raise TypeError('Source for {!r} not found'.format(object))
inspect.getfile = new_getfile

import inspect
from IPython.core.magics.code import extract_symbols

def get_code(obj):

    try:
        return inspect.getsource(obj)
    except Exception as e:
        # in case we're looking for a class that was generated within a
        # Jupyter notebook we use the following
        cell_code = "".join(inspect.linecache.getlines(new_getfile(obj)))
        class_code = extract_symbols(cell_code, obj.__name__)[0][0]
        return class_code

In [5]:
class ClassCode():
    def __init__(self, entrypoint: str, return_type: str, code: str) -> None:
        self.code = code
        self.entrypoint = entrypoint
        self.return_type = return_type
        
    def send(self, client):
        tree = ast.parse(self.code)
        client.secure_exec(entrypoint=self.entrypoint, return_type=self.return_type, ast_tree=tree)
        print(self.usage_code())
        
    def usage_code(self) -> str:
        return (
            "Give this to your DS:\n"
            + "ptr = client.run(\n"
            + f"    entrypoint=\"{self.entrypoint}\",\n"
            + f"    return_type=\"{self.return_type}\"\n"
            +")"
        )

    def __repr__(self):
        return self.code

In [6]:
@biwrap.biwrap
def sendable(func_or_class, return_type="syft.lib.python._SyNone"):
    strip_decorator = re.sub(r"@sendable.+\n","", get_code(func_or_class))
    return ClassCode(entrypoint = func_or_class.__name__, return_type=return_type, code=strip_decorator)

In [12]:
@sendable(return_type="pycolab.engine.Engine")
def make_game():
    from pycolab import ascii_art
    from pycolab import things
    
    ART = ["1@"]
    
    class RollingDrape(things.Drape):
        def update(self, *args):
            pass


    class SlidingSprite(things.Sprite):
        def __init__(self, *args):
            self._visible = False
            pass

        def update(self, *args):
            pass


    return ascii_art.ascii_art_to_game(
        ART,
        what_lies_beneath=" ",
        sprites={
            "1": ascii_art.Partial(SlidingSprite, 0),
        },
        drapes={"@": RollingDrape},
        z_order="1@",
    )

In [13]:
make_game.send(root_client)

Remote Secure Exec
Compiling...
... compiled!
Accepting, code bound at entry point successfully!
Give this to your DS:
ptr = client.run(
    entrypoint="make_game",
    return_type="pycolab.engine.Engine"
)


In [14]:
game = sy.sandbox.make_game()

In [15]:
def run(client, entrypoint: str, return_type: str):
    # make a real existing pointer
    import syft
    ref = syft.lib_ast.query(return_type)
    ptr_type = ref.pointer_type
    ptr = ptr_type(
        client=client,
        id_at_location=UID(),
        object_type=return_type,
    )
    
    action = RunFunctionOrConstructorAction(
        path=f"syft.sandbox.{entrypoint}",
        args=tuple(),
        kwargs={},
        id_at_location=ptr.id_at_location,
        address=client.address,
        msg_id=UID(),
    )

    client.send_immediate_msg_without_reply(msg=action)

    return ptr

In [16]:
ptr = run(root_client,
    entrypoint="make_game",
    return_type="pycolab.engine.Engine"
)

calling into module ['syft', 'sandbox', 'make_game'] 1 None
calling into module ['syft', 'sandbox', 'make_game'] 2 None


In [17]:
root_client.store.pandas

,ID,Tags,Description,object_type
0,<UID: febebce52f2c4fff93f400ea317b4e31>,[],,<class 'pycolab.engine.Engine'>


In [20]:
start_tuple_ptr = ptr.its_showtime()

calling into module ['pycolab', 'engine', 'Engine', 'its_showtime'] 1 None
calling into module ['pycolab', 'engine', 'Engine', 'its_showtime'] 2 None


In [25]:
board_ptr = start_tuple_ptr[0].resolve_pointer_type()
reward_ptr = start_tuple_ptr[1].resolve_pointer_type()
discount_ptr = start_tuple_ptr[2].resolve_pointer_type()

calling into module ['syft', 'lib', 'python', 'Tuple', '__getitem__'] 1 None
calling into module ['syft', 'lib', 'python', 'Tuple', '__getitem__'] 2 None
calling into module ['syft', 'lib', 'python', 'Tuple', '__getitem__'] 3 None
calling into module ['syft', 'lib', 'python', 'Tuple', '__getitem__'] 1 None
calling into module ['syft', 'lib', 'python', 'Tuple', '__getitem__'] 2 None
calling into module ['syft', 'lib', 'python', 'Tuple', '__getitem__'] 3 None
calling into module ['syft', 'lib', 'python', 'Tuple', '__getitem__'] 1 None
calling into module ['syft', 'lib', 'python', 'Tuple', '__getitem__'] 2 None
calling into module ['syft', 'lib', 'python', 'Tuple', '__getitem__'] 3 None


In [29]:
board_ptr.get(delete_obj=False)

Observation(board=array([[32, 64]], dtype=uint8), layers={'1': array([[0, 0]], dtype=uint8), ' ': array([[1, 0]], dtype=uint8), '@': array([[0, 1]], dtype=uint8)})

In [30]:
reward_ptr.get(delete_obj=False)

In [31]:
discount_ptr.get(delete_obj=False)

1.0

In [ ]:
game_tuple_ptr = ptr.play("0")

In [46]:
def resolve_game(tuple_ptr):
    return resolve(tuple_ptr, 3)

In [49]:
def resolve(tuple_ptr, size=1):
    resolved = []
    for i in range(size):
        resolved.append(tuple_ptr[i].resolve_pointer_type())
    return tuple(resolved)

In [50]:
game_tuple_ptrs = resolve_game(ptr.play("0"))

calling into module ['pycolab', 'engine', 'Engine', 'play'] 1 None
calling into module ['pycolab', 'engine', 'Engine', 'play'] 2 None
calling into module ['syft', 'lib', 'python', 'Tuple', '__getitem__'] 1 None
calling into module ['syft', 'lib', 'python', 'Tuple', '__getitem__'] 2 None
calling into module ['syft', 'lib', 'python', 'Tuple', '__getitem__'] 3 None
calling into module ['syft', 'lib', 'python', 'Tuple', '__getitem__'] 1 None
calling into module ['syft', 'lib', 'python', 'Tuple', '__getitem__'] 2 None
calling into module ['syft', 'lib', 'python', 'Tuple', '__getitem__'] 3 None
calling into module ['syft', 'lib', 'python', 'Tuple', '__getitem__'] 1 None
calling into module ['syft', 'lib', 'python', 'Tuple', '__getitem__'] 2 None
calling into module ['syft', 'lib', 'python', 'Tuple', '__getitem__'] 3 None


In [51]:
game_tuple_ptrs

(<syft.proxy.pycolab.rendering.ObservationPointer at 0x13efcca30>,
 <syft.proxy.syft.lib.python.FloatPointer at 0x13efc2370>)

In [ ]:

#     # syft.lib.python.Tuple
#     fake_pointer = Pointer(id_at_location=action.id_at_location, client=client)
#     fake_pointer.path_and_name=action.path
#     fake_pointer.__class__.__module__ = f"syft.proxy.syft.lib.python"
#     fake_pointer.__class__.__name__ = f"TuplePointer"
#     fake_pointer.__class__.__qualname__ = f"TuplePointer"
#     return fake_pointer

In [ ]:
def run_on(pointer, method: str, args = [], kwargs = {}):
    action = RunClassMethodAction(
        path=f"{pointer.path_and_name}.{method}",
        _self=pointer,
        args=args,
        kwargs=kwargs,
        id_at_location=UID(),
        address=pointer.client.address,
        msg_id=UID(),
    )
    pointer.client.send_immediate_msg_without_reply(msg=action)

    fake_pointer = Pointer(id_at_location=action.id_at_location, client=pointer.client)
    fake_pointer.path_and_name = action.path
#     fake_pointer.__class__.__module__ = f"syft.proxy.{'.'.join(module_parts)}"
#     fake_pointer.__class__.__name__ = f"{klass_name}Pointer"
#     fake_pointer.__class__.__qualname__ = f"{klass_name}Pointer"
    return fake_pointer

In [ ]:
# def init_class(client, path: str):
#     action = RunFunctionOrConstructorAction(
#         path="syft.sandbox.mylib.Test",
#         args=tuple(),
#         kwargs={},
#         id_at_location=UID(),
#         address=client.address,
#         msg_id=UID(),
#     )

#     client.send_immediate_msg_without_reply(msg=action)

#     module_parts = path.split(".")
#     klass_name = module_parts.pop()
#     # syft.proxy.torch.TensorPointer
#     fake_pointer = Pointer(id_at_location=action.id_at_location, client=client)
#     fake_pointer.path_and_name=action.path
#     fake_pointer.__class__.__module__ = f"syft.proxy.{'.'.join(module_parts)}"
#     fake_pointer.__class__.__name__ = f"{klass_name}Pointer"
#     fake_pointer.__class__.__qualname__ = f"{klass_name}Pointer"
#     return fake_pointer

# def run_on(pointer, method: str, args = [], kwargs = {}):
#     action = RunClassMethodAction(
#         path=f"{pointer.path_and_name}.{method}",
#         _self=pointer,
#         args=args,
#         kwargs=kwargs,
#         id_at_location=UID(),
#         address=pointer.client.address,
#         msg_id=UID(),
#     )
#     pointer.client.send_immediate_msg_without_reply(msg=action)

#     fake_pointer = Pointer(id_at_location=action.id_at_location, client=pointer.client)
#     fake_pointer.path_and_name = action.path
# #     fake_pointer.__class__.__module__ = f"syft.proxy.{'.'.join(module_parts)}"
# #     fake_pointer.__class__.__name__ = f"{klass_name}Pointer"
# #     fake_pointer.__class__.__qualname__ = f"{klass_name}Pointer"
#     return fake_pointer

In [ ]:
test_ptr = init_class(root_client, "syft.sandbox.mylib.Test")
hello_ptr = run_on(test_ptr, "hello")

In [ ]:
# run_on(test_ptr, "__init__")

In [ ]:
hello_ptr.get()